In [1]:
import pandas as pd
import os
import re
import json
from collections import OrderedDict
import googlemaps

In [2]:
%run ../config.py

In [3]:
def geocode(gmaps_client, end):
    
    geocode_result = gmaps_client.geocode(end)
    
    return geocode_result

def salvar_json_end(dados, version, path_salvar = None):
    
    if path_salvar is None:
        path_salvar = '../Dados_gerados/geocode'
        if not os.path.exists(path_salvar):
            os.mkdir(path_salvar)
    
    fname = os.path.join(path_salvar, f'enderecos_georref_{version}.json')
    
    with open(fname, 'w') as f:
        json.dump(dados, f)

def get_end_salvo(versao, path = None):

    if path is None:
        path = '../Dados_gerados/geocode'
    if os.path.exists(path):
        pat = 'enderecos_georref_\d*.json'
        files = [os.path.join(path, file) for file in os.listdir(path)
                if re.match(pat, file)]

        for f in files:
            versao_file = os.path.split(f)[-1].split('_')[-1].split('.')[0]
            if str(versao) == versao_file:
                with open(f) as file:
                    return json.load(file)


    return None

In [4]:
def pegar_todos_end(gmaps_client, enderecos, versao, salvar = True, path = None):
    
    versao_existente = get_end_salvo(versao, path)
    
    if versao_existente:
        print(f'Retornando versão {versao} já salva')
        return versao_existente
    
    
    end_geocode = {}
    count_requisi = 0
    total_requisi = len(enderecos)
    for end in enderecos:
        end_geocode[end] = geocode(gmaps_client, end)
        count_requisi+=1
        if not count_requisi%10:
            print(f'{count_requisi} requisiçoes feitas.')
            print(f'{round(count_requisi/total_requisi, 2)*100}% do total')
    if salvar:
        salvar_json_end(end_geocode, versao, path)
    
    return end_geocode

In [5]:
def get_end_n_georref():
    
    with open('../Dados_gerados/ends_n_georref.json') as f:
        ends = json.load(f)
        
    return ends

In [6]:
def dfs_georref():
    
    prop = pd.read_excel('../Dados_gerados/prop_georref_cep.xlsx')
    contrib = pd.read_excel('../Dados_gerados/contrib_georref_cep.xlsx')
    
    return prop, contrib

In [7]:
def get_component_by_type(comp, type_name):
    
    if type_name in comp['types']:
        return comp['long_name']
    else:
        return None

def parse_address_comps(address):
    
    de_para = OrderedDict(
        {
            'bairro' : 'sublocality_level_1',
            'cidade' : 'administrative_area_level_2',
            'estado' : 'administrative_area_level_1',
            'pais' : 'country',
            'CEP_Google' : 'postal_code'
            
        }
    )
    
    dados = {}
    for nom_meu, nom_google in de_para.items():
        #nao consegui pensar em uma otimizacao, infelizmente,
        # mas a ordem tende a seguir a do de-para
        for component in address['address_components']:
            parsed = get_component_by_type(component, nom_google)
            if parsed:
                dados[nom_meu] = parsed
                break
        else:
            dados[nom_meu] = None
    
    return dados

def parse_address(address):
    
    dados = {}
    dados.update(parse_address_comps(address))
    dados['endereco_completo'] = address['formatted_address']
    dados['tipo_endereco'] = '; '.join(address['types'])
    dados['latitude'] = address['geometry']['location']['lat']
    dados['longitude'] = address['geometry']['location']['lng']
    dados['tipo_localizacao'] = address['geometry']['location_type']
    
    return dados

def registro_vazio():
    
    return {
                'bairro' : None,
                'cidade' : None,
                'estado' : None,
                'pais' : None,
                'endereco_completo' : None,
                'tipo_endereco' : None,
                'latitude' : None,
                'longitude' : None,
                'tipo_localizacao' : None,
                'CEP_Google' : None
            }

def repescar_ceps(df, address_col, end_n_georref, gmaps, versao):
    
    #filtra o df para os registros nao georreferenciados
    df_n_georref =df[df[address_col].isin(end_n_georref)].copy()
    #pega os ceps unicos
    ceps = df_n_georref['CEP'].unique()
    #remove nulos
    ceps = [cep for cep in ceps if cep]
    #geocodifica em cima desses ceps
    new_geocode = pegar_todos_end(gmaps, ceps, versao)
    
    return new_geocode

def repescagem_georref(df, address_col, end_n_georref, gmaps, versao_geo):
    
    new_geocode = repescar_ceps(df, address_col, end_n_georref, 
                               gmaps, versao_geo)
    #nao vou modificar inplace
    df = df.copy().reset_index(drop=True)
    
    #agora coloca os dados no dataframe original
    for i, row in df.iterrows():
        
        if row[address_col] not in end_n_georref:
            #nao faz nada
            continue
        #esta nos nao georrefs, entao faz algo
        else:
            cep = row['CEP']
            #se CEP é nulo, registro é nulo
            if not cep:
                dados_geo = registro_vazio()
            else:
                dados_geo_raw = new_geocode[cep]
                if len(dados_geo_raw) < 1:
                    #nao achou nada
                    dados_geo = registro_vazio()
                elif len(dados_geo_raw) > 1:
                    #só pra ver quando vem mais de um
                    print(dados_geo_raw)
                    dados_geo = parse_address(dados_geo_raw[0])
                else:
                    #veio um só, entao pega ele que é o primeiro
                    dados_geo = parse_address(dados_geo_raw[0])
        
            #pegamos os dados para os dois casos, então:
            for col, val in dados_geo.items():
                df.loc[i, col] = val
    
    return df

In [8]:
def check_cep(row):
    
    if pd.isnull(row['CEP']) and pd.isnull(row['CEP_Google']):
        return True
    else:
        check = row['CEP'] == row['CEP_Google']
        return check

In [9]:
prop, contrib = dfs_georref()
end_n_georref = get_end_n_georref()
gmaps = googlemaps.Client(google_api_key)

In [10]:
prop_repesc = repescagem_georref(prop, 'Endereco', end_n_georref, gmaps, versao_geo = 11)
contrib_repesc = repescagem_georref(contrib, 'Endereço', end_n_georref, gmaps, versao_geo = 12)

Retornando versão 11 já salva
Retornando versão 12 já salva


In [11]:
prop_repesc['check_cep_repesc'] = prop_repesc.apply(check_cep, axis = 1)
contrib_repesc['check_cep_repesc'] = contrib_repesc.apply(check_cep, axis = 1)

In [12]:
prop_repesc['check_cep_repesc'].sum()/len(prop_repesc)

0.9479166666666666

In [13]:
prop_repesc['check_cep'].sum()/len(prop_repesc)

0.8113425925925926

In [14]:
contrib_repesc['check_cep_repesc'].sum()/len(contrib_repesc)

0.976867151354924

In [15]:
contrib_repesc['check_cep'].sum()/len(contrib_repesc)

0.9345670852610707

Ótimos resultados. Vamos salvar e explorar um pouco os dados

In [16]:
prop_repesc.to_excel('../Dados_gerados/prop_georref_final.xlsx')
contrib_repesc.to_excel('../Dados_gerados/contrib_georref_final.xlsx')

In [17]:
contrib_repesc['tipo_localizacao'].unique()

array(['ROOFTOP', 'APPROXIMATE', 'RANGE_INTERPOLATED', nan,
       'GEOMETRIC_CENTER', None], dtype=object)

In [18]:
contrib_repesc[contrib_repesc['tipo_localizacao']=='GEOMETRIC_CENTER']

,Unnamed: 0,ID,Numeração,Título,Texto,Opção escolhida,Sugestão,Usuário,Email,Endereço,...,pais,endereco_completo,tipo_endereco,latitude,longitude,tipo_localizacao,CEP,CEP_Google,check_cep,check_cep_repesc
567,567,11401,2.2.2001.,Meta 30,Meta 30: Construir 14 novos piscinões. Indicad...,Revisar,Há outras maneiras mais modernas do que os pis...,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
593,593,11402,2.2.1.4.,Iniciativa D,d. Retirar 820.000 toneladas de detritos dos p...,Revisar,Por isso que penso que piscinão não é uma boa ...,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1102,1102,11403,4.4.2001.,Meta 59,Meta 59: Estruturar 400 hortas urbanas e ações...,Manter,Muito bom!!! espero que de certo,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1156,1156,11404,5.1.2001.,Meta 60,Meta 60: Implantar 8 novos parques municipais....,Manter,"Otimo, espero que o parque Alto da Boa Vista e...",Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1176,1176,11405,5.1.1.3.,Iniciativa C,c. Elaborar os Planos de Gestão dos novos parq...,Manter,Como seria esse plano de gestão?,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1184,1184,11406,5.1.2.1.,Iniciativa A,a. Implantar 2 Unidades de Conservação;,Manter,Onde seria essa unidade de conservação? Temos ...,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1188,1188,11407,5.1.2.3.,Iniciativa C,c. Revisar e/ou elaborar os Planos de Manejo d...,Manter,Como seria feito isso?,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1214,1214,11408,5.1.3.2.,Iniciativa B,b. Criar o Sistema de Gestão da Arborização;,Revisar,Temos um problema muuuuuito sério na cidade to...,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1355,1355,11409,5.2.2001.,Meta 67,"Meta 67: Reduzir em 600 mil toneladas, a quant...",Manter,muito importante,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True
1372,1372,11410,5.2.1.2.,Iniciativa B,b. Permitir o recolhimento de 4 outros tipos d...,Manter,NaN,Cecilia oliveira,ceciliaagco@gmail.com,Rua Dez de Novembro 695 São Paulo SP,...,Brazil,"R. Dez de Novembro - Chácara Flora, São Paulo ...",route,-23.648505,-46.685411,GEOMETRIC_CENTER,04644-080,04644-080,True,True


In [19]:
prop_repesc[prop_repesc['tipo_localizacao']=='GEOMETRIC_CENTER']

,Unnamed: 0,ID,Data de criação,Autor,Email,Endereco,Proposta,Resumo da proposta,Texto da proposta,Link do vídeo externo,...,pais,endereco_completo,tipo_endereco,latitude,longitude,tipo_localizacao,CEP,CEP_Google,check_cep,check_cep_repesc
1,1,23,08/04/2021,andersonbalmeida,andersonbalmeida@gmail.com,Rua Fani Lerner 30 São Paulo SP,Cinturão de proteção ambiental Jaragua,Impedir o desmatamento e exploração imobiliári...,Impedir o desmatamento e exploração imobiliári...,NaN,...,Brazil,"R. Fani Lerner, 30 - Jaragua, São Paulo - SP, ...",premise,-23.434769,-46.740747,GEOMETRIC_CENTER,02995-260,02995-260,True,True
2,2,24,08/04/2021,andersonbalmeida,andersonbalmeida@gmail.com,Rua Fani Lerner 30 São Paulo SP,sistema de transporte público rápido na região...,Implantar sistema de transporte público rápido...,A região mais periférica da região noroeste (B...,NaN,...,Brazil,"R. Fani Lerner, 30 - Jaragua, São Paulo - SP, ...",premise,-23.434769,-46.740747,GEOMETRIC_CENTER,02995-260,02995-260,True,True
740,740,765,07/05/2021,Inst. Blandina Meirelles,cca@blandinameirelles.org.br,Rua Francisco Soave 0 São Paulo SP,Implantação do CRAS Rio Pequeno,Implementação de um CRAS no distrito Rio Peque...,Tendo em vista a alta vulnerabilidade do distr...,NaN,...,Brazil,"R. Francisco Soave - Rio Pequeno, São Paulo - ...",route,-23.566308,-46.757882,GEOMETRIC_CENTER,05386-000,05386-000,True,True


In [20]:
prop_repesc[prop_repesc['tipo_localizacao']=='RANGE_INTERPOLATED']

,Unnamed: 0,ID,Data de criação,Autor,Email,Endereco,Proposta,Resumo da proposta,Texto da proposta,Link do vídeo externo,...,pais,endereco_completo,tipo_endereco,latitude,longitude,tipo_localizacao,CEP,CEP_Google,check_cep,check_cep_repesc
22,22,44,12/04/2021,Anderson Sutherland,anderson.sutherland@gmail.com,Rua Paulo Mendes Rodrigues 20 São Paulo SP,Mobilidade Ativa,Modal Ativo - Uma Cidade Mais Humana - Criar C...,1 Criar ciclos(vias e ou faixas) nas avenidas ...,NaN,...,Brazil,"R. Paulo Mendes Rodrigues, 20 - Conj. Hab. Jov...",street_address,-23.454000,-46.577950,RANGE_INTERPOLATED,02281-263,02281-263,True,True
68,68,90,19/04/2021,Mayara Torres,mayara576@hotmail.com,Rua Nanquim 91 São Paulo SP,Construção do CEU Cidade Ademar,O processo de desapropriação do imóvel destina...,Há anos os moradores da Cidade Ademar reivindi...,https://www.facebook.com/watch/?v=251883748530245,...,Brazil,"R. Nanquim, 91 - Jardim Pedreira, São Paulo - ...",street_address,-23.693513,-46.666974,RANGE_INTERPOLATED,04462-050,04462-050,True,True
70,70,92,19/04/2021,Mayara Torres,mayara576@hotmail.com,Rua Nanquim 91 São Paulo SP,Instalação de um Fab Lab Livre SP na região da...,.,O FAB LAB LIVRE SP é uma rede de laboratórios ...,NaN,...,Brazil,"R. Nanquim, 91 - Jardim Pedreira, São Paulo - ...",street_address,-23.693513,-46.666974,RANGE_INTERPOLATED,04462-050,04462-050,True,True
132,132,154,21/04/2021,Emerbonus,emerbonus@bol.com.br,Rua Luís Gonzaga de Sylos 94 São Paulo SP,Aumentar o efetivo da GCM De SP E valorização ...,Aumentar o efetivo em no mínimo 1500 Gcms pois...,Tendo em vista que sua grande maioria tem em m...,NaN,...,Brazil,"Rua Luís Gonzaga de Sylos, 94 - Jardim Aricand...",street_address,-23.566732,-46.508072,RANGE_INTERPOLATED,03455-070,03455-070,True,True
195,195,217,24/04/2021,ASSOCIAÇÃO ESPORTIVA JARDIM SINHÁ,elga-oliveira@hotmail.com,Rua Henry Fuseli 70 Viela são pedro São Paulo SP,Transformação do Campo de futebol Jardim Sinhá...,Atender melhor nossos moradores com aulas de f...,"Criação de um CDC (Clube da Comunidade), na ún...",NaN,...,Brazil,"Rua Henry Fuseli, 70 - Parque dos Bancarios, S...",street_address,-23.591336,-46.510063,RANGE_INTERPOLATED,03923-030,03923-030,True,True
197,197,219,24/04/2021,CiclistaBinotti,bikerzl.brasil@gmail.com,Rua da Tropicália 287 Casa 3 São Paulo SP,Canal de acesso direto a estruturas cicloviári...,Atualmente o aplicativo da prefeitura não poss...,NaN,NaN,...,Brazil,"R. da Tropicália, 287 - Casa 3 - Jardim Pedro ...",subpremise,-23.504638,-46.465775,RANGE_INTERPOLATED,08061-010,08061-010,True,True
212,212,234,24/04/2021,ASSOCIAÇÃO ESPORTIVA JARDIM SINHÁ,elga-oliveira@hotmail.com,Rua Henry Fuseli 70 Viela são pedro São Paulo SP,INVESTIMENTO EM NOSSA ÁREA E REGIÃO DO SAPOPEM...,"COMUNIDADE DO JARDIM SINHÁ, MAIS INVESTIMENTOS...",SER MELHOR ATENDIDOS PELOS ORGÃOS PÚBLICOS A N...,NaN,...,Brazil,"Rua Henry Fuseli, 70 - Parque dos Bancarios, S...",street_address,-23.591336,-46.510063,RANGE_INTERPOLATED,03923-030,03923-030,True,True
267,267,289,26/04/2021,Ghe Santos II,consultorlgbt@gmail.com,Rua Lagoa da Cruz 17 São Paulo SP,Terminal de ônibus Kemel,Beneficiar a população doKemel Existem varias ...,CONSTRUÇÃO DO TERMINAL KEMEL PARA BENEFICIAR A...,NaN,...,Brazil,"R. Lagoa da Cruz, 17 - Itaim Paulista, São Pau...",street_address,-23.497551,-46.375150,RANGE_INTERPOLATED,08122-300,08122-300,True,True
271,271,293,26/04/2021,Ghe Santos II,consultorlgbt@gmail.com,Rua Lagoa da Cruz 17 São Paulo SP,Canalização do Córrego Ribeirão Tres Pontes,Canalização do Córrego Ribeirão Três Pontes (T...,Canalização do Córrego Ribeirão Tres Pontes (T...,NaN,...,Brazil,"R. Lagoa da Cruz, 17 - Itaim Paulista, São Pau...",street_address,-23.497551,-46.375150,RANGE_INTERPOLATED,08122-300,08122-300,True,True
275,275,297,26/04/2021,Ghe Santos II,consultorlgbt@gmail.com,Rua Lagoa da Cruz 17 São Paulo SP,Construção da UPA AVANÇADA - SOS UPA Kemel - E...,Construção da UPA KEMEL - Encosta Norte benefi...,BAIRRO KEMEL consta da (ubs - ke